In [1]:
import os
import re
import sys

import MeCab

In [2]:
nm = MeCab.Tagger('-Owakati')

# txt 파일을 전부 읽어들여 카테고리를 지정하고 정제한다.
files = [f for f in os.listdir('./dataset/raw_datas') if f.split('.')[1] == 'txt']
category = {'humanities': [], 'economy': [], 'science': []}
cleaned_data = []

In [3]:
def clean(raw_text):
    """
    . 기준으로 한문장씩 분할하여 쓴다.
    """
    text = raw_text.replace('\n', '').replace('\u3000', '')
    text = text.replace('.', '.\n').split('\n')

    return [nm.parse(t).split(' ') for t in text if t]

for f in files:
        name = f.split('.')[0]
        print(name)
        with open('./dataset/raw_datas/'+str(f), 'r') as f:
            clean_text = clean(f.read())
            category[name] = clean_text
            cleaned_data += clean_text

economy
humanities
science


In [4]:
def make_stopdic(lines):
    """
    공백으로 구분된 문장의 모음인 텍스트 리스트로부터 스톱워드 사전을 작성한다.
    """
    calc_words = {}
    for line in lines:
        for word in line:
            if word in calc_words:
                calc_words[str(word)] += 1
            else:
                calc_words[str(word)] = 1
    sorted_stop = sorted(calc_words.items(), key=lambda x:x[1], reverse=True)
    print("sorted_stop:"+str(len(sorted_stop)))
    freq_num = int(len(sorted_stop)*0.03)
    n = 0
    stop_words = []
    print("freq_num:"+str(freq_num))
    for data in sorted_stop:
        stop_words.append(str(data[0]))
        #print('High frequency word: ',str(data[0]), str(data[1]))
        n += 1
        if n > freq_num:
            break
    # 작성한 스톱워드 사전을 저장한다. 
    with open('./origin_stopwords.txt', 'w') as f:
        f.write('\n'.join(stop_words))

make_stopdic(cleaned_data)
print("cleaned_data:"+str(len(cleaned_data)))

sorted_stop:13298
freq_num:398
cleaned_data:8529


In [5]:
# 스톱워드 리스트를 작성한다. 
stop_data = {}
for f in files:
    with open('./dataset/raw_datas/'+str(f)) as f1:
        print(f)
        nm = MeCab.Tagger('-Owakati')
        text = nm.parse(f1.read()).split(' ')
        for word in text:
            if word in stop_data:
                stop_data[str(word)] += 1
            else:
                stop_data[str(word)] = 1

economy.txt
humanities.txt
science.txt


In [6]:
sorted_stop = sorted(stop_data.items(), key=lambda x:x[1], reverse=True)

print(len(sorted_stop))

freq_num = int(len(sorted_stop)*0.03)
n = 0
stop_words = []
print(freq_num)
for data in sorted_stop:
    stop_words.append(str(data[0]))
    print('High frequency word: ',str(data[0]), str(data[1]))
    n+= 1
    if n > freq_num:
        break

# 작성한 스톱워드의 사전을 저장한다. 
with open('./origin_stopwords.txt', 'w') as f:
    f.write('\n'.join(stop_words))

13295
398
High frequency word:  . 8478
High frequency word:  이 7317
High frequency word:  는 7287
High frequency word:  을 5680
High frequency word:  하 4945
High frequency word:  , 4487
High frequency word:  의 4197
High frequency word:  에 4002
High frequency word:  은 3721
High frequency word:  를 3572
High frequency word:  다 3346
High frequency word:  가 2840
High frequency word:  있 2804
High frequency word:  한 2252
High frequency word:  고 2218
High frequency word:  수 1792
High frequency word:  것 1789
High frequency word:  에서 1753
High frequency word:  적 1715
High frequency word:  으로 1692
High frequency word:  들 1652
High frequency word:  도 1430
High frequency word:  ) 1397
High frequency word:  ( 1384
High frequency word:  지 1297
High frequency word:  게 1215
High frequency word:  기 1212
High frequency word:  로 1201
High frequency word:  할 1180
High frequency word:  과 1120
High frequency word:  되 1105
High frequency word:  면 875
High frequency word:  인 869
High frequency word:  어 835
High 

In [7]:
def stopword_bydic(text):
    """
    사전에 따라 스톱워드를 제거한다. 
    """
    # 읽어들일 스톱워드 사전을 지정한다. 
    with open('./origin_stopwords.txt') as f:
        data = f.read()
        stopwords = data.split('\n')
    lines = []
    for line in text:
        words = []
        for word in line:
            if word not in stopwords:
                words.append(word)    
        lines.append(words)
    return lines

In [8]:
sum_lines = len(stopword_bydic(cleaned_data))
sum_words = []
for line in stopword_bydic(cleaned_data):
    for word in line:
        if word not in sum_words:
            sum_words.append(word)
print('sum_lines: ', sum_lines)
print('sum_words: ', len(sum_words))
with open('origin_words.txt', 'w') as f:
    f.write('\n'.join(sum_words))
cleaned_data = []
for category, data in category.items():
    data = stopword_bydic(data)
    for line in data:
        #if len(line) > 2:
        cleaned_data.append(category+','+' '.join(line))

sum_lines:  8529
sum_words:  12899


In [9]:
# 작성한 코퍼스를 저장한다.
with open('./dataset/corpus.csv', 'w') as f:
    f.write(''.join(cleaned_data))